In [95]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tabulate import tabulate
from datetime import datetime
from pathlib import Path
import numpy as np
from io import StringIO
import os
import re
import pandas as pd
import requests
import json
import time


In [96]:
def get_ticker_information(ticker):
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

    ticker_information = requests.get(f"https://efts.sec.gov/LATEST/search-index?keysTyped={ticker}", headers=headers)

    response = ticker_information.content.decode()#["hits"]["hits"][0]["_source"]["entity"]
    entity_name = json.loads(response)["hits"]["hits"][0]["_source"]["entity"]
    
    return entity_name

In [103]:
class Consolidated_Schedule_Investments():
    def __init__(self, ticker, entity_name, url):
        self.ticker = ticker
        self.entity_name = entity_name
        self.url = url
    
    def save_File(self, data, filename):

        with pd.ExcelWriter(filename) as writer:
            for sheet_name, df in data:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # path = Path(filename)
        # df = pd.DataFrame(data)
        # df.to_csv(path, index=False)
    
    def get_table_of_url(self, driver):

        urls = []
        reporting_dates = []

        table = driver.find_element(By.XPATH, '//*[@id="hits"]/table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for i in range(1, len(rows)):
            cell = driver.find_element(By.XPATH, f'//*[@id="hits"]/table/tbody/tr[{i}]/td[1]/a')
            data_adsh = cell.get_attribute('data-adsh')
            data_adsh = data_adsh.replace('-','')
            data_file_name = cell.get_attribute('data-file-name')
            cell_reporting_date = driver.find_element(By.XPATH, f'//*[@id="hits"]/table/tbody/tr[{i}]/td[3]').text
            unformated_date = datetime.strptime(cell_reporting_date, "%Y-%m-%d")
            reporting_dates.append(cell_reporting_date)

            url = f"https://www.sec.gov/Archives/edgar/data/{self.ticker}/{data_adsh}/{data_file_name}"
            urls.append(url)
        
        return urls, reporting_dates
    
    # Function to extract the target table based on specific heuristics
    def extract_table_selenium(self, driver, n_rows, name_first_row):
        
        tables = driver.find_elements(By.TAG_NAME, "table")
        candidate_tables = []

        for table in tables:
            # print(repr(table.text))
            # Heuristics to identify the correct table
            # Example: Select tables with more than 2 rows and 2 columns
            table
            rows = table.find_elements(By.TAG_NAME,'tr')
            if len(rows) > n_rows:
                i = 0
                while True:
                    if rows[i].text != '\n\n\n\n':
                        cols = rows[i].find_elements(By.TAG_NAME, 'td')
                        break

                    i +=1
                
                if name_first_row in rows[0].text:
                    candidate_tables.append(table.text)
        
        # If multiple tables match, refine selection logic
        if len(candidate_tables) >= 1:
            # Example: Further refine based on specific row or column content
            choices = ["Principal Ammount", "Value", "Cost"]
            for table in candidate_tables:
                if any(x in str(table) for x in choices):
                    continue
                else:
                    remove_idx = candidate_tables.index(table)
                    candidate_tables.pop(remove_idx)
            return candidate_tables  # Fallback to the first candidate

        return None

    
    # Function to extract the target table based on specific heuristics
    def extract_table(self, soup, n_rows, name_first_row):
        tables = soup.find_all('table')
        candidate_tables = []

        for table in tables:
            # print(repr(table.text))
            # Heuristics to identify the correct table
            # Example: Select tables with more than 2 rows and 2 columns
            rows = table.find_all('tr')
            if len(rows) > n_rows:
                i = 0
                while True:
                    if rows[i].text != '\n\n\n\n':
                        cols = rows[i].find_all(['td'])
                        break

                    i +=1
                
                if name_first_row in rows[1].text:
                    candidate_tables.append(table)
        
        # If multiple tables match, refine selection logic
        if len(candidate_tables) > 1:
            # Example: Further refine based on specific row or column content
            choices = ["Co-Investments", "Primary Private Investment Funds", "Secondary Private Investment Funds"]
            for table in candidate_tables:
                if any(x in str(table) for x in choices):
                    continue
                else:
                    remove_idx = candidate_tables.index(table)
                    candidate_tables.pop(remove_idx)
            return candidate_tables  # Fallback to the first candidate

        return None
    
    def amg_pantheon_fund(self):

        dataframes = []

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.implicitly_wait(5)
        driver.maximize_window()
        driver.get(self.url)
        time.sleep(3)

        

        urls_dates = self.get_table_of_url(driver)
        dates = urls_dates[1]
        urls = urls_dates[0]
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
        
        for j in range(len(urls)):
            while True:
                r = requests.get(urls[j], headers=headers)
                if r.status_code == 200:
                    soup = BeautifulSoup(r.text, 'html.parser')
                    break

            target_table = self.extract_table(soup, 10, "Initial")
            firts_text = False
            full_text_table = ""

            for table in target_table:
                if target_table:
                    texts = table.text
                    texts = texts.replace('\xa0', '')
                    texts = texts.replace('\u2003', '')
                    texts = texts.strip('\n\t ')
                    texts = texts.replace('\n \n', '\n\n')
                    texts = texts.replace('Initial\nAcquisition', 'Security\tInitial Acquisition')
                    texts = texts.replace('Initial Acquisition\nDate', 'Security\tInitial Acquisition Date')
                    texts = texts.replace('InitialAcquisitionDate', 'Security\tInitial Acquisition Date')
                    texts = texts.replace('PercentofNet\nAssets', 'Percent of Net Assets')
                    texts = re.sub(r'(?<=\w)(\n)+Secondary', ' \nSecondary',texts)
                    texts = texts.replace(' (continued)', '')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', '\t\t\t\t\n')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', '\n')
                    texts = re.sub(r'(?<=%)\n+(?=\$)', '\t', texts)
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n', '\t\t')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n\n', '\t\t\t\t\n')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n', '\t\t')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n', '\t\t\t\t\n')
                    texts = re.sub(r'(?<=[a-zA-Z])\n+(?=[0-9])', '\t', texts)
                    texts = re.sub(r'Net Assets\n+', 'Net Assets\t\t', texts)
                    texts = re.sub(r'Other Assets, less Liabilities\n+', 'Other Assets, less Liabilities\t\t', texts)
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n', ' \n')
                    texts = texts.replace('\n\n\n\n\n\n\n\n', '\n')
                    texts = re.sub(r'\n+(?=\d)', '\t', texts)
                    texts = re.sub(r'(?<=\S)\s*\n{7}(?=\s*\d)', '\t', texts)
                    texts = texts.replace('\n\n\n\n\n\n\n', '\n')
                    texts = texts.replace('\n\n\n\n\n\n', '\n')
                    texts = texts.replace('\n\n\n\n', '\t')
                    texts = texts.replace("\n\n\n Net Assets", "\n Net Assets")
                    texts = texts.replace("\n\n Net Assets", "\n Net Assets")
                    texts = texts.replace('\n\n\n', '\t')
                    texts = texts.replace('\n\nSe', '\n')
                    texts = texts.replace('\n\n', '\t')
                    texts = re.sub(r'\n%', '%', texts)
                    texts = re.sub(r'(\S)\n(\S)', r'\1 \2', texts)
                    texts = re.sub(r'(\S)\n(\s*\()', r'\1 \2', texts)
                    texts = re.sub(r'(?<=\S)\n(?=\d)|(?<=\d)\n(?=\S)', '', texts)
                    texts = re.sub(r'^[ \t]+(?=\S)', '', texts, flags=re.MULTILINE)
                    texts = re.sub(r'\t{5,} ?', '\n', texts)
                    texts = re.sub(r'(?<=\d)\t{3,}', '\n', texts)
                    texts = texts.replace('\t Total', '\n Total')
                    texts = texts.replace('$', '')
                    texts = re.sub(r'(\(\w\),)+\*', '', texts)
                    texts = texts.replace('*', '')
                    texts = texts.replace('\t\t', '\t')
                    texts = texts.replace("\t\t\n", "\n")
                    texts = re.sub(r'(?<=\d)\t\n', '\n', texts)

                    if firts_text:
                        full_text_table += f"\n{texts}"
                    else:
                        firts_text= True
                        full_text_table = texts


                    # print(repr(f"\n----------------------------------------------------------------------------------\n{texts}"))
                    # print(f"\n----------------------------------------------------------------------------------\n{texts}")
            # print(full_text_table)
            result = StringIO(full_text_table)
            try:
                df = pd.read_csv(result, sep='\t', header=0, index_col=False)
            except:
                continue
            
            if len(df.columns) > 4:
                df.columns = ["Security", "Initial Acquisition Date", "Shares", "Value", "Percent of Net Assets"]
                df = df[(df.Security != "Security")&(df["Initial Acquisition Date"] != "Initial Acquisition Date")&(df.Shares != "Shares")&(df.Value != "Value")&(df["Percent of Net Assets"] != "Percent of Net Assets (1)")]
            else:
                df.columns = ["Security", "Initial Acquisition Date", "Shares", "Value"]
                df = df[(df.Security != "Security")&(df["Initial Acquisition Date"] != "Initial Acquisition Date")&(df.Shares != "Shares")&(df.Value != "Value")]
            
            
            df["Report Date"] = [dates[j] for i in range(len(df['Initial Acquisition Date'].values))]

            # Create a new column for sections
            df['Type'] = np.nan

            # Forward fill the section column for non-NaN 'Security' rows
            section = None
            for i, row in df.iterrows():
                try:
                    if pd.isna(row['Initial Acquisition Date']) and pd.isna(row['Shares']) and pd.isna(row['Value']) and pd.isna(row['Percent of Net Assets']):
                        section = row['Security']
                except KeyError:
                    if pd.isna(row['Initial Acquisition Date']) and pd.isna(row['Shares']) and pd.isna(row['Value']):
                        section = row['Security']
                
                df.at[i, 'Type'] = section

            try:
                # Drop the rows that are sections
                df = df.dropna(subset=['Initial Acquisition Date', 'Shares', 'Value', 'Percent of Net Assets'])
            except KeyError:
                # Drop the rows that are sections
                df = df.dropna(subset=['Initial Acquisition Date', 'Shares', 'Value'])

            # Reset the index of the DataFrame
            df = df.reset_index(drop=True)

            df['Value'] = df['Value'].str.replace(',', '')

            # Define the pattern to search for (partial match)
            pattern = re.compile(r'^Investment is issued in a private placement offering')
            original_price_text = soup.find(lambda tag: tag.name == "p" and pattern.search(tag.text)).text

            # Regular expression pattern to match numbers with commas
            pattern = r'\$\d{1,3}(?:,\d{3})*'
            # Find all matches in the text
            matches = re.findall(pattern, original_price_text)
            if matches:
                matches = matches[:-1]
                matches = [x.replace('$', '').replace(',', '') for x in matches]
            
            else:
                date = df['Report Date'][0]
                date_obt = datetime.strptime(date, "%Y-%m-%d")

                pattern = re.compile(r'^\s*As of {0}\xa0{1}, {2}, the aggregate cost'.format(date_obt.strftime("%B"), date_obt.strftime("%d"), date_obt.strftime("%Y")))
                soup = BeautifulSoup(r.text, 'html.parser')
                
                original_price_text = soup.find(lambda tag: tag.name == "td" and pattern.search(tag.text))
                if original_price_text == None:
                    pattern = re.compile(r'^\n*\s*As of {0}\xa0{1}, {2}, the aggregate cost'.format(date_obt.strftime("%B"), date_obt.strftime("%d"), date_obt.strftime("%Y")))
                    driver.get(urls[4])
                    text_tag = driver.find_element(By.TAG_NAME, 'text')
                    p_tags = text_tag.find_elements(By.TAG_NAME, 'p')
                    
                    for paragraph in p_tags:
                        pattern = re.compile(r'^\n*\s*As of {0} {1}, {2}, the aggregate cost'.format(date_obt.strftime("%B"), date_obt.strftime("%d"), date_obt.strftime("%Y")))
                        result = pattern.search(paragraph.text)
                        if result:
                            original_price_text = paragraph.text
                            break
                else:
                    original_price_text = original_price_text.text
                # Regular expression pattern to match numbers with commas
                pattern = r'\$\d{1,3}(?:,\d{3})*'
                # Find all matches in the text
                matches = re.findall(pattern, original_price_text)
                matches = matches[:-1]
                matches = [x.replace('$', '').replace(',', '') for x in matches]

            df['Original Value'] = matches

            try:
                columns = ["Type", "Security", "Initial Acquisition Date", "Report Date", "Shares", "Original Value", "Value", "Percent of Net Assets"]
                df = df[columns]
            except KeyError:
                columns = ["Type", "Security", "Initial Acquisition Date", "Report Date", "Shares", "Original Value", "Value"]
                df = df[columns]
            
            df_obj = df.select_dtypes('object')

            df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

            df['Report Date'] = pd.to_datetime(df['Report Date'], format="%Y-%m-%d")
            df['Report Date'] = df['Report Date'].apply(lambda x: x.strftime('%m/%d/%Y'))
            df['Initial Acquisition Date'] = pd.to_datetime(df['Initial Acquisition Date'], format='%m/%d/%Y')
            df['Initial Acquisition Date'] = df['Initial Acquisition Date'].apply(lambda x: x.strftime('%m/%d/%Y'))

            date = datetime.strptime(df['Report Date'][0], '%m/%d/%Y')
            date = date.strftime('%Y_%m_%d')
            print(date)

            # dataframes.append((f'{date}', df))
        
        
            
            df.to_excel(f"Private Credit Funds.xlsx", sheet_name=f"{date}", index=False)
    
    def first_trust_private_fund(self):
        
        dataframes = []

        chrome_install = ChromeDriverManager().install()

        folder = os.path.dirname(chrome_install)
        # chromedriver_path = os.path.join(folder, "chromedriver.exe")
        
        #MacOS
        folder = folder.replace("127.0.6533.99", "126.0.6478.126")
        chromedriver_path = os.path.join(folder, "chromedriver")


        driver = webdriver.Chrome(service=ChromeService(chromedriver_path))
        driver.implicitly_wait(5)
        driver.maximize_window()
        driver.get(self.url)
        time.sleep(3)

        

        urls_dates = self.get_table_of_url(driver)
        dates = urls_dates[1]
        urls = urls_dates[0]
        print(urls[0])
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

        for j in range(len(urls)):
            
            driver.get(urls[j])

            target_table = self.extract_table_selenium(driver, 7, "Principal")
            if not target_table:
                target_table = self.extract_table_selenium(driver, 4, "Principal")
            # print(target_table[0])
            firts_text = False
            full_text_table = ""
            df = None
            first = True

            # Current section tracker
            current_section = ""

            for table in target_table:
                if target_table:
                    texts = table
                    texts = texts.replace('\u2007', '')
                    texts = texts.replace('\u2009', '')
                    texts = texts.replace('Principal\nAmount1', 'Principal Amount')
                    texts = texts.replace('Principal\nAmount', 'Principal Amount')
                    texts = texts.replace('Principal Amount           Value', 'Principal Amount\nValue')
                    texts = texts.replace('Principal Amount         Value  ', 'Principal Amount\nValue')
                    texts = texts.replace(' Palmer', '\nPalmer')
                    texts = texts.replace('Inc.9', 'Inc.')
                    texts = texts.replace('\npoints', ' points')
                    texts = texts.replace('\nbasis', ' basis')
                    texts = texts.replace('basis\n', 'basis ')
                    texts = texts.replace('Ares Capital 2023-1', 'Ares Capital')
                    texts = texts.replace('Trust 2021-1', 'Trust')
                    texts = texts.replace('8 Stronghold Digital Mining, Inc.–Class A*,1 35', '8\nStronghold Digital Mining, Inc.–Class A')
                    texts = texts.replace('),\n', '), ')

                    # Split the data into lines
                    lines = texts.split('\n')
                    lines.pop(0)
                    lines.pop(0)

                    # Initialize lists for each column
                    principal_amount = []
                    value = []
                    security = []
                    company = []
                    type = []
                    
                    f_value = False
                    
                    # Process each line
                    for i in range(0, len(lines)):
                        
                        variable = lines[i]
                        variable_no = lines[i-1]

                        if 'ASSET' in lines[i] or 'BANK LOANS' in lines[i] or "CLOSED-END" in lines[i] or "COLLATERALIZED LOAN" in lines[i] or "ELECTRIC-GENERATION" in lines[i]:
                            if "Continued" not in lines[i] and "TOTAL" not in lines[i]:
                                current_section = lines[i].strip()

                            continue

                        is_value = re.findall(r'\d{1,3}(?:,\d{3})+', lines[i])
                        is_company = re.findall(r'\b(?!\d+|.*\d+)[A-Za-z/&.,\s]+\b(?<![\d/])', lines[i])
                        is_security =  re.findall(r'.*?\d{1,2}\.\d{1,3}%.*?\d{1,2}\/\d{1,2}\/\d{4}', lines[i])

                        # print(f"{len(is_value) == 1} {"TOTAL" not in lines[i-1]} {"TOTAL" not in lines[i-2]}")

                        if is_value and len(is_value) == 1 and "TOTAL" not in lines[i-1] and (i < 2 or "TOTAL" not in lines[i-2]):
                            
                            if "ELECTRIC-GENERATION" in current_section:
                                break

                            is_company_alt = re.sub(r"\d{1,3}(?:,\d{3})+", "", lines[i]).strip()
                            if is_company_alt != '' and is_company_alt != '$' and 'Class' not in is_company_alt and not is_security and len(is_company_alt)>5:
                                if '$' in is_company_alt:
                                    is_company_alt = is_company_alt.replace('$', '').strip()
                                
                                company.append(is_company_alt)

                            if is_security:
                                security.append(is_security[0].strip())

                            if not f_value:
                                f_value = True
                                principal_amount.append(is_value[0].strip())

                            elif f_value:
                                f_value = False

                                value.append(is_value[0].strip())
                                if current_section != "":
                                    type.append(current_section)
                        
                        elif is_value and len(is_value) == 2 and "TOTAL" not in lines[i-1] and (i < 2 or "TOTAL" not in lines[i-2]):
                            f_value = False

                            principal_amount.append(is_value[0].strip())
                            value.append(is_value[1].strip())

                            is_security =  re.findall(r'\d{1,2}\.\d{1,3}%.*?\d{1,2}\/\d{1,2}\/\d{4}', lines[i])
                            
                            security.append(is_security[0].strip())

                            is_company = re.findall(r'\b(?!\d+|.*\d+)[A-Za-z/&.,\s]+\b(?<![\d/])', lines[i-1])

                            if not is_company:
                                company.append(company[-1])
                            
                            if current_section != "":
                                type.append(current_section)

                        elif is_security:
                            
                            if "ELECTRIC-GENERATION" in current_section:
                                break

                            if "CLOSED-END" in current_section:
                                security.append('')
                                continue
                            security.append(is_security[0].strip())
                            
                            is_company = re.findall(r'\b(?!\d+|.*\d+)[A-Za-z/&.,\s]+\b(?<![\d/])', lines[i-1])

                            if "Series" in lines[i] and not is_company:
                                try:
                                    company.append(company[-1])
                                except IndexError:
                                    company.append(df["Company"].iloc[-1])
                        
                        elif is_company and 'Number' not in lines[i] and 'of Shares' not in lines[i] and 'Principal' not in lines[i]:

                            if "ELECTRIC-GENERATION" in current_section:
                                break
                            
                            company.append(lines[i].strip())

                            if "CLOSED-END" in current_section:
                                security.append('')
                    
                    # print(len(principal_amount))
                    # print(len(value))
                    # print(len(security))
                    # print(len(company))
                    # print(len(type))

                    # print(principal_amount)
                    # print(value)
                    # print(security)
                    # print(company)
                    # print(type)

                    if first:
                        # Create a DataFrame
                        df = pd.DataFrame({
                            'Principal Amount': principal_amount,
                            'Value': value,
                            'Security': security,
                            'Company': company,
                            'Type': type
                        })
                        first = False
                    else:
                        # Create a DataFrame
                        df_new = pd.DataFrame({
                            'Principal Amount': principal_amount,
                            'Value': value,
                            'Security': security,
                            'Company': company,
                            'Type': type
                        })
                        df = pd.concat([df, df_new], ignore_index=True)
            
            df['Value'] = df['Value'].str.replace(',', '')
            df['Value'] = df['Value'].astype('int32')
            df['Principal Amount'] = df['Principal Amount'].str.replace(',', '')
            df['Principal Amount'] = df['Principal Amount'].astype('int32')

            df["Report Date"] = [dates[j] for i in range(len(df['Principal Amount'].values))]
            df['Report Date'] = pd.to_datetime(df['Report Date'], format="%Y-%m-%d")
            df['Report Date'] = df['Report Date'].apply(lambda x: x.strftime('%m/%d/%Y'))

            date = datetime.strptime(df['Report Date'][0], '%m/%d/%Y')
            date = date.strftime('%Y_%m_%d')

            # Initialize counter
            counter = 1
            new_date = date

            # Loop until we find a unique date
            while any(df[0] == new_date for df in dataframes):
                new_date = f"{date} ({counter})"
                counter += 1

            if date ==  new_date:
                dataframes.append((f'{date}', df))
            else:
                dataframes.append((f'{new_date}', df))

        with pd.ExcelWriter('Private Credit Funds.xlsx') as writer:
            for sheet_name, df in dataframes:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        driver.quit()

    def ares_private_markets_fund(self):
        dataframes = []

        chrome_install = ChromeDriverManager().install()

        folder = os.path.dirname(chrome_install)
        chromedriver_path = os.path.join(folder, "chromedriver.exe")
        
        #MacOS
        # folder = folder.replace("127.0.6533.99", "126.0.6478.126")
        # chromedriver_path = os.path.join(folder, "chromedriver")


        driver = webdriver.Chrome(service=ChromeService(chromedriver_path))
        driver.implicitly_wait(5)
        driver.maximize_window()
        driver.get(self.url)
        time.sleep(3)

        

        urls_dates = self.get_table_of_url(driver)
        dates = urls_dates[1]
        urls = urls_dates[0]
        print(urls[0])

        for j in range(len(urls)):
        
            driver.get(urls[j])

            target_table = self.extract_table_selenium(driver, 7, "Geographic")
            if not target_table:
                target_table = self.extract_table_selenium(driver, 4, "Geographic")
            
            df = None
            first = True

            # Current section tracker
            current_section = ""

            for table in target_table:
                if target_table:
                        texts = table
                        texts = texts.replace('\u200c', '')
                        texts = texts.replace('(a)', '')
                        texts = texts.replace('(b)', '')
                        texts = texts.replace('(c)', '')
                        texts = texts.replace('(d)', '')
                        texts = texts.replace('(e)', '')
                        texts = texts.replace('(f)', '')
                        texts = texts.replace('(g)', '')
                        texts = texts.replace(' %', '%')


                        # Split the data into lines
                        lines = texts.split('\n')
                        # lines.pop(0)
                        # lines.pop(0)

                        while 'Direct Investments/Co-Investments' not in lines[0] and 'Primary Investments' not in lines[0] and "Secondary Investments" not in lines[0]:
                            lines.pop(0)
                        
                        # print(repr(f"\n----------------------------------------------------------------------------------\n{texts}"))

                        # Initialize lists for each column
                        cost = []
                        value = []
                        percentage = []
                        security = []
                        region = []
                        adquisition_date = []
                        type = []
                        
                        # Process each line
                        for i in range(0, len(lines)):
                            
                            # variable = lines[i]
                            # variable_no = lines[i-1]

                            if 'Direct Investments/Co-Investments' in lines[i] or 'Primary Investments' in lines[i] or "Secondary Investments" in lines[i]:
                                if "continued" not in lines[i] and "Total" not in lines[i]:
                                    current_section = lines[i].strip()

                                continue
                            
                            is_value = re.findall(r'\d{1,3}(?:,\d{3})+', lines[i])
                            # is_security = re.findall(r'\b(?!\d+|.*\d+)[A-Za-z/&.,\s-]+\b(?<![\d/])', lines[i])
                            is_security = re.findall(r'(?:[A-Za-z\d/&.,()\s-]+(?<![\d/])\b)', lines[i])
                            is_adate = re.findall(r'\d{1,2}\/\d{1,2}\/\d{4}', lines[i])
                            is_percentage = re.findall(r'\d{1,2}\.\d{1,3}\s*%', lines[i])

                            if len(is_value) == 2 and 'Total' not in lines[i]:
                                value.append(is_value[1])
                                cost.append(is_value[0])
                                type.append(current_section)
                            
                            elif len(is_value) == 1 and 'Total' not in lines[i] and '–' in lines[i]:
                                value.append(is_value[0])
                                cost.append(np.nan)
                                type.append(current_section)

                            elif len(is_value) <= 1 and 'Total' not in lines[i]:
                                
                                if "Short-Term Investments" in lines[i]:
                                    break

                                is_value_alt = re.findall(r'\d{1,3}(?:,\d{3})*', lines[i])
                                # print(is_value_alt)

                                try:
                                    index_value = is_value_alt.index(is_value[0])

                                    if index_value == 4:
                                        value.append(is_value_alt[5])
                                        cost.append(is_value[0])
                                        type.append(current_section)

                                    else:
                                        value.append(is_value[0])
                                        cost.append(is_value_alt[4])
                                        type.append(current_section)

                                except IndexError:
                                    value.append(is_value_alt[5])
                                    cost.append(is_value_alt[4])
                                    type.append(current_section)




                            if is_security and 'Total' not in lines[i]:
                                # Additional step to split based on multiple spaces
                                is_security = [match.strip() for match in re.split(r'\s{2,}', is_security[0]) if match.strip()]
                                
                                security.append(is_security[0])
                                region.append(is_security[1])

                            if is_adate:
                                adquisition_date.append(is_adate[0])
                            
                            if is_percentage:
                                percentage.append(is_percentage[0])

                        # print(len(security))
                        # print(len(region))
                        # print(len(type))
                        # print(len(adquisition_date))
                        # print(len(cost))
                        # print(len(value))
                        # print(len(percentage))


                        # print(security)
                        # print(region)
                        # print(type)
                        # print(adquisition_date)
                        # print(cost)
                        # print(value)
                        # print(percentage)

                        if first:
                            # Create a DataFrame
                            df = pd.DataFrame({
                                'Security': security,
                                'Geographic Region': region,
                                'Type': type,
                                'Adqusition Date': adquisition_date,
                                'Cost': cost,
                                'Fair Value': value,
                                'Percentage of Net Assets': percentage

                            })
                            first = False
                        else:
                            # Create a DataFrame
                            df_new = pd.DataFrame({
                                'Security': security,
                                'Geographic Region': region,
                                'Type': type,
                                'Adqusition Date': adquisition_date,
                                'Cost': cost,
                                'Fair Value': value,
                                'Percentage of Net Assets': percentage

                            })
                            df = pd.concat([df, df_new], ignore_index=True)                    
                
            df['Cost'] = df['Cost'].str.replace(',', '')
            df['Cost'] = df['Cost'].astype('Int32')
            df['Fair Value'] = df['Fair Value'].str.replace(',', '')
            df['Fair Value'] = df['Fair Value'].astype('Int32')

            df["Report Date"] = [dates[j] for i in range(len(df['Cost'].values))]
            df['Report Date'] = pd.to_datetime(df['Report Date'], format="%Y-%m-%d")
            df['Report Date'] = df['Report Date'].apply(lambda x: x.strftime('%m/%d/%Y'))

            columns =  ['Security', 'Geographic Region', 'Type', 'Adqusition Date', 'Report Date', 'Cost', 'Fair Value', 'Percentage of Net Assets']
            df = df[columns]

            date = datetime.strptime(df['Report Date'][0], '%m/%d/%Y')
            date = date.strftime('%Y_%m_%d')

            # Initialize counter
            counter = 1
            new_date = date

            # Loop until we find a unique date
            while any(df[0] == new_date for df in dataframes):
                new_date = f"{date} ({counter})"
                counter += 1

            if date ==  new_date:
                dataframes.append((f'{date}', df))
            else:
                dataframes.append((f'{new_date}', df))

        with pd.ExcelWriter('Private Credit Funds.xlsx') as writer:
            for sheet_name, df in dataframes:
                df.to_excel(writer, sheet_name=sheet_name, index=False)

        driver.quit()

    def fs_mvp_private_markets_fund(self):
        
        dataframes = []

        chrome_install = ChromeDriverManager().install()

        folder = os.path.dirname(chrome_install)
        chromedriver_path = os.path.join(folder, "chromedriver.exe")
        
        #MacOS
        # folder = folder.replace("127.0.6533.99", "126.0.6478.126")
        # chromedriver_path = os.path.join(folder, "chromedriver")


        driver = webdriver.Chrome(service=ChromeService(chromedriver_path))
        driver.implicitly_wait(5)
        driver.maximize_window()
        driver.get(self.url)
        time.sleep(3)

        

        urls_dates = self.get_table_of_url(driver)
        dates = urls_dates[1]
        urls = urls_dates[0]
        print(urls[0])

        for j in range(len(urls)):
            
            print(f'Table {j}\n')
            driver.get(urls[j])

            target_table = self.extract_table_selenium(driver, 2, "Cost")
            if not target_table:
                target_table = self.extract_table_selenium(driver, 3, "Description")

            df = None
            first = True

            # Current section tracker
            current_section = ""

            for table in target_table:
                if target_table:
                    texts = table
                    texts = texts.replace('\u200c', '')
                    texts = texts.replace('(a)', '')
                    texts = texts.replace('(b)', '')
                    texts = texts.replace('(c)', '')
                    texts = texts.replace('(d)', '')
                    texts = texts.replace('(e)', '')
                    texts = texts.replace('(f)', '')
                    texts = texts.replace('(g)', '')
                    texts = texts.replace('(h)', '')
                    texts = texts.replace('(i)', '')
                    texts = texts.replace(',\n', ', ')
                    texts = texts.replace(' 0 ', ' 000,000')
                    texts = texts.replace('US\nL', 'US L')
                    texts = re.sub(r'\,\s\(', ' (', texts)
                    texts = re.sub(r'\([0-9]{1,2}\.?[0-9]{0,2}%\)', '', texts)
                    texts = re.sub(r'\)\n', ')', texts)
                    texts = re.sub(r'\+\s*\n', '+ ', texts)
                    texts = re.sub(r'\%\s*\n', '% ', texts)
                    if 'Short-Term' in current_section:
                        texts = re.sub(r'\s*\n\d{1,2}\.?\d{0,3}\s*%', r'\1%', texts)
                    texts = re.sub(r'\s*\n\d{1,2}\.?\d{0,3}\s*%', r'\0%', texts)
                    texts = re.sub(r'\s*\n\s*\bSquare\b', ' Square', texts)
                    texts = re.sub(r'\s*\n\s*\bFund\b', ' Fund', texts)
                    texts = re.sub(r'\s*\n\s*\bClass\b', ' Class', texts)
                    texts = texts.replace(' Goldman', '\nGoldman')
                    texts = texts.replace(' Vanguard', '\nVanguard')
                    texts = texts.replace('\nL.P.', ' L.P.')
                    texts = texts.replace('\nStrategic', ' Strategic')
                    texts = texts.replace('\nBlocker', ' Blocker')
                    texts = texts.replace('\nInvestment', ' Investment')
                    texts = texts.replace('\nPartners', ' Partners')
                    texts = texts.replace('\nSeries', ' Series')
                    texts = texts.replace('\nUnits', ' Units')
                    texts = texts.replace(' \nDescription', 'Description')
                    texts = texts.replace('\nFair Value', 'Fair Value')
                    texts = texts.replace('%                           Total', '%\nTotal')
                    texts = re.sub(r'\s*\n\s*\bMarket\b', ' Market', texts)
                    texts = texts.replace('Percentage of\nNet Assets', 'Percentage of Net Assets')
                    texts = texts.replace('Direct Equity   \nAcquisition \nDate \n  Cost     Fair Value  ', 'Acquisition Date   Cost     Fair Value\nDirect Equity')
                    texts = texts.replace('Primary Investments   \nAcquisition \nDate\n  Cost     Fair Value  ', 'Acquisition Date  Cost     Fair Value  \nPrimary Investments')
                    texts = texts.replace('Secondary Investments   \nAcquisition \nDate \n  Cost     Fair Value  ', 'Acquisition Date   Cost     Fair Value  \nSecondary Investments ')
                    texts = re.sub(r'\n?\bAcquisition\b\s*\n\s*\bDate\b', 'Acquisition Date', texts)
                    texts = re.sub(r'\n\s*\bCost\b', ' Cost', texts)
                    texts = texts.replace('Short-Term Investments    Cost     Fair Value', 'Cost     Fair Value\nShort-Term Investments')
                    texts = texts.replace('Short-Term Investments    Cost   Fair Value   Percentage of Net Assets', 'Cost   Fair Value   Percentage of Net Assets\nShort-Term Investments')


                    # Split the data into lines
                    lines = texts.split('\n')
                    # lines.pop(0)
                    # lines.pop(0)

                    while 'Direct Credit' not in lines[0] and 'Direct Equity' not in lines[0] and "Secondary Investment" not in lines[0] and "Primary Investment" not in lines[0] and 'Short-Term' not in lines[0]:
                        
                        if 'continued' in lines[0]:
                            lines.pop(0)
                            break

                        if len(lines) <= 5:
                            lines.pop(0)
                            break

                        lines.pop(0)
                    
                    # print(repr(f"\n----------------------------------------------------------------------------------\n{texts}"))

                    # Initialize lists for each column
                    description = []
                    principal = []
                    maturity_date = []
                    adquisition_date = []
                    cost = []
                    fair_value = []
                    percentage_net_assets = []
                    type = []

                    short_term_security = ''

                    # Process each line
                    for i in range(0, len(lines)):
                        
                        variable = lines[i]
                        variable_no = lines[i-1]

                        if 'Direct Credit' in lines[i] or 'Direct Equity' in lines[i] or "Secondary Investment" in lines[i] or "Primary Investment" in lines[i] or 'Short-Term' in lines[i]:
                            if "continued" not in lines[i] and "Total" not in lines[i]:
                                current_section = lines[i].strip()

                            continue

                        if 'Liabilities' in lines[i] or 'Other Assets In Excess of' in lines[i]:
                            break
                        
                        is_value = re.findall(r'\d{1,3}(?:,\d{3})+', lines[i])
                        # is_security = re.search(r'[A-Za-z /-]+([,]?\s[A-Za-z./]+[0-9]*)*\s\([0-9A-Za-z]+(\s[A-Za-z]+)+\s\+\s[0-9]{1,2}\.[0-9]{1,2}%\)', lines[i])
                        is_security = re.search(r'[A-Za-z /-]+([,]?\s[A-Za-z0-9./]+[0-9]*)*\s\(?[0-9A-Za-z]+(\s[A-Za-z]+)+\s\+\s[0-9]{1,2}\.?[0-9]{0,2}%([,]\s\d{1,2}\/\d{1,2}\/\d{4})?\)?(\s\bdue\b\s*\d{1,2}\/\d{1,2}\/\d{4})?', lines[i])
                        # is_security = re.search(r'[A-Za-z /-]+([,]?\s[A-Za-z0-9./]+[0-9]*(?:,\d{3})*)*(\s\(?[0-9A-Za-z]+(\s[A-Za-z]+)+\s\+\s[0-9]{1,2}\.?[0-9]{0,2}%([,]\s\d{1,2}\/\d{1,2}\/\d{4})?\))?(\s\bdue\b\s*\d{1,2}\/\d{1,2}\/\d{4})?', lines[i])
                        is_security_alt = re.search(r'([A-Za-z&]+(\s[0-9]*)?\s)*([A-Za-z\-,]+\s)+[A-Za-z.]*[0-9.%]*', lines[i])
                        is_adate = re.findall(r'\d{1,2}\/\d{1,2}\/\d{4}', lines[i])
                        is_percentage = re.findall(r'\d{1,2}\.?\d{0,3}\s*%', lines[i])

                        if not is_value and not is_security and not is_security_alt and not is_adate and not is_percentage:
                            continue
                        
                        if len(is_value)>= 2 and ('Total' not in lines[i] or 'Totalmed' in lines[i]):
                            if len(is_value) == 2:
                                principal.append(None)
                                cost.append(is_value[0])
                                fair_value.append(is_value[1])
                                type.append(current_section)
                            
                            elif len(is_value) == 3:
                                principal.append(is_value[0])
                                cost.append(is_value[1])
                                fair_value.append(is_value[2])
                                type.append(current_section)
                            
                        if is_security or is_security_alt and ('Total' not in lines[i] or 'Totalmed' in lines[i]):
                            if is_security:
                                description.append(is_security.group())
                            
                            elif is_security_alt and not is_security: #and 'Short-Term' not in current_section:
                                description.append(is_security_alt.group())
                            
                            # elif is_security_alt and not is_security and 'Short-Term' in current_section:
                            #     if '%' in is_security_alt.group() and 'Class' not in is_security_alt.group():
                            #         description.append(is_security_alt.group())
                        
                        if is_adate and len(is_value) ==3 and ('Total' not in lines[i] or 'Totalmed' in lines[i]):
                            
                            # if is_security_alt and not is_security and 'Short-Term' not in current_section:
                            maturity_date.append(is_adate[0])
                            adquisition_date.append(None)
                            
                            # elif is_security_alt and not is_security and 'Short-Term' in current_section:
                            #     if '%' in is_security_alt.group() and 'Class' not in is_security_alt.group():
                            #         maturity_date.append(is_adate[0])
                            #         adquisition_date.append(None)
                            
                            # else:
                            #     maturity_date.append(is_adate[0])
                            #     adquisition_date.append(None)

                        elif is_adate and len(is_value) ==2 and ('Total' not in lines[i] or 'Totalmed' in lines[i]):
                            
                            # if is_security_alt and not is_security and 'Short-Term' not in current_section:
                            maturity_date.append(None)
                            adquisition_date.append(is_adate[0])
                            
                            # elif is_security_alt and not is_security and 'Short-Term' in current_section:
                            #     if '%' in is_security_alt.group() and 'Class' not in is_security_alt.group():
                            #         maturity_date.append(None)
                            #         adquisition_date.append(is_adate[0])
                            
                            # else:
                            #     maturity_date.append(None)
                            #     adquisition_date.append(is_adate[0])
                        
                        elif not is_adate and ('Total' not in lines[i] or 'Totalmed' in lines[i]):
                            
                            # if is_security_alt and not is_security and 'Short-Term' not in current_section:
                            maturity_date.append(None)
                            adquisition_date.append(None)
                            
                            # elif is_security_alt and not is_security and 'Short-Term' in current_section:
                            #     if '%' in is_security_alt.group() and 'Class' not in is_security_alt.group():
                            #         maturity_date.append(None)
                            #         adquisition_date.append(None)
                            
                            # else:
                            #     maturity_date.append(None)
                            #     adquisition_date.append(None)
                                    
                        if is_percentage and ('Total' not in lines[i] or 'Totalmed' in lines[i]):

                            if len(is_percentage) == 2:
                                percentage_net_assets.append(is_percentage[1])

                                # if 'Short-Term' in current_section and not is_security_alt:
                                #     short_term_security += is_percentage[0]
                                #     description.append(short_term_security)
                                #     maturity_date.append(None)
                                #     adquisition_date.append(None)
                                #     short_term_security = ''
                                # elif 'Short-Term' in current_section and is_security_alt:
                                #     short_term_security += is_security_alt.group()
                                #     description.append(short_term_security)
                                #     maturity_date.append(None)
                                #     adquisition_date.append(None)
                                #     short_term_security = ''

                            elif len(is_percentage) == 1:
                                percentage_net_assets.append(is_percentage[0])

                        elif not is_value and not is_security_alt and not is_security:
                            continue
                        # else:
                        #     if 'Short-Term' in current_section:
                        #         if is_security_alt and not is_value:
                        #             short_term_security += is_security_alt.group()
                                
                        #     else:
                        elif 'Total' not in lines[i] or 'Totalmed' in lines[i]:
                            percentage_net_assets.append(None)

                    print(len(description))
                    print(len(principal))
                    print(len(type))
                    print(len(maturity_date))
                    print(len(adquisition_date))
                    print(len(cost))
                    print(len(fair_value))
                    print(len(percentage_net_assets))


                    print(description)
                    print(principal)
                    print(type)
                    print(maturity_date)
                    print(adquisition_date)
                    print(cost)
                    print(fair_value)
                    print(percentage_net_assets)

                    if first:
                        # Create a DataFrame
                        df = pd.DataFrame({
                            'Description': description,
                            'Principal': principal,
                            'Type': type,
                            'Maturity Date': maturity_date,
                            'Adqusition Date': adquisition_date,
                            'Cost': cost,
                            'Fair Value': fair_value,
                            'Percentage of Net Assets': percentage_net_assets

                        })
                        first = False
                    else:
                        # Create a DataFrame
                        df_new = pd.DataFrame({
                            'Description': description,
                            'Principal': principal,
                            'Type': type,
                            'Maturity Date': maturity_date,
                            'Adqusition Date': adquisition_date,
                            'Cost': cost,
                            'Fair Value': fair_value,
                            'Percentage of Net Assets': percentage_net_assets
                        })
                        df = pd.concat([df, df_new], ignore_index=True)
            
            df['Cost'] = df['Cost'].str.replace(',', '')
            df['Cost'] = df['Cost'].astype('Int32')
            df['Fair Value'] = df['Fair Value'].str.replace(',', '')
            df['Fair Value'] = df['Fair Value'].astype('Int32')

            df["Report Date"] = [dates[j] for i in range(len(df['Cost'].values))]
            df['Report Date'] = pd.to_datetime(df['Report Date'], format="%Y-%m-%d")
            df['Report Date'] = df['Report Date'].apply(lambda x: x.strftime('%m/%d/%Y'))

            columns =  ['Description', 'Type', 'Maturity Date', 'Adqusition Date', 'Report Date', 'Principal', 'Cost', 'Fair Value', 'Percentage of Net Assets']
            df = df[columns]

            date = datetime.strptime(df['Report Date'][0], '%m/%d/%Y')
            date = date.strftime('%Y_%m_%d')

            # Initialize counter
            counter = 1
            new_date = date

            # Loop until we find a unique date
            while any(df[0] == new_date for df in dataframes):
                new_date = f"{date} ({counter})"
                counter += 1

            if date ==  new_date:
                dataframes.append((f'{date}', df))
            else:
                dataframes.append((f'{new_date}', df))

        with pd.ExcelWriter('Private Credit Funds.xlsx') as writer:
            for sheet_name, df in dataframes:
                df.to_excel(writer, sheet_name=sheet_name, index=False)

        driver.quit()


In [104]:
ticker = 1856156
entity = get_ticker_information(ticker)
url = f"https://www.sec.gov/edgar/search/#/category=custom&ciks=000{ticker}&entityName={entity}(CIK 000{ticker})&forms=N-CSR,N-CSRS"
# print(url)


# Consolidated_Schedule_Investments(ticker, entity, url).amg_pantheon_fund()
# Consolidated_Schedule_Investments(ticker, entity, url).first_trust_private_fund()
Consolidated_Schedule_Investments(ticker, entity, url).fs_mvp_private_markets_fund()

https://www.sec.gov/Archives/edgar/data/1856156/000139834424011419/fp0088065-1_ncsr.htm
Table 0

18
18
18
18
18
18
18
18
['Aero Operating, LLC (3M US SOFR + 9.00%)', 'Beacon Oral Specialist (3M US SOFR + 6.00%)', 'Beta Plus Technologies, Inc., TL (3M US SOFR + 5.75%)', 'CMS Group Holdings, LLC (3M US SOFR + 5.5%)', 'EIS Legacy, LLC TL (3M US SOFR + 5.75%)', 'Erie Construction Mid-West, LLC (3M US SOFR + 4.75%)', 'ETE Intermediate II, LLC TL (3M US SOFR + 6.50%)', 'MDME Holdings, LLC (3M US SOFR + 6.25%)', 'MDME Incremental T/L (3M US SOFR + 6.25%)', 'NAS, LLC (3M US SOFR + 6.50%)', 'Netrix, LLC (3M US SOFR + 7.83%)', 'North Acquisition LLC (3M US SOFR + 6.75%)', 'Orthodontic Partner LLC DDTL4 (3M US SOFR + 6.50%)', 'Orthodontic Partners, LLC DDTL (3M US SOFR + 6.50%)', 'PLA Buyer, LLC (3M US SOFR + 6.90%)', 'PLA Buyer, LLC DDTL (3M US SOFR + 6.90%)', 'PLA Revolver, LLC DDTL (3M US SOFR + 6.90%)', 'Road Tested Parts, LLC TL (3M US SOFR + 6.50%)']
['6,669,740', '7,389,881', '4,937,343', 